# TensorFlow practice 1

In [27]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
import tensorflow as tf
import datetime
import pandas as pd
import numpy as np

In [29]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [30]:
# this why input_shap=(28, 28)
x_train.shape

(60000, 28, 28)

In [31]:
x_train[0,0,]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
# find out the unique values
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [44]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=70, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/70
1/1 [==============================] - 1s 674ms/step - loss: 2.1029 - accuracy: 0.2188 - val_loss: 2.1119 - val_accuracy: 0.4375
Epoch 2/70
1/1 [==============================] - 0s 445ms/step - loss: 1.5952 - accuracy: 0.6875 - val_loss: 2.0893 - val_accuracy: 0.4375
Epoch 3/70
1/1 [==============================] - 0s 25ms/step - loss: 1.2145 - accuracy: 0.8750 - val_loss: 2.0951 - val_accuracy: 0.3750
Epoch 4/70
1/1 [==============================] - 0s 24ms/step - loss: 0.8962 - accuracy: 0.8750 - val_loss: 2.1079 - val_accuracy: 0.4375
Epoch 5/70
1/1 [==============================] - 0s 25ms/step - loss: 0.6669 - accuracy: 0.9688 - val_loss: 2.1253 - val_accuracy: 0.4375
Epoch 6/70
1/1 [==============================] - 0s 32ms/step - loss: 0.4477 - accuracy: 0.9688 - val_loss: 2.1413 - val_accuracy: 0.5000
Epoch 7/70
1/1 [==============================] - 0s 26ms/step - loss: 0.3308 - accuracy: 1.0000 - val_loss: 2.1634 - val_accuracy: 0.5000
Epoch 8/70
1/1 [=========

1/1 [==============================] - 0s 26ms/step - loss: 7.5569e-04 - accuracy: 1.0000 - val_loss: 3.2140 - val_accuracy: 0.4375
Epoch 60/70
1/1 [==============================] - 0s 27ms/step - loss: 7.1656e-04 - accuracy: 1.0000 - val_loss: 3.2176 - val_accuracy: 0.4375
Epoch 61/70
1/1 [==============================] - 0s 27ms/step - loss: 7.4085e-04 - accuracy: 1.0000 - val_loss: 3.2210 - val_accuracy: 0.4375
Epoch 62/70
1/1 [==============================] - 0s 27ms/step - loss: 6.2405e-04 - accuracy: 1.0000 - val_loss: 3.2244 - val_accuracy: 0.4375
Epoch 63/70
1/1 [==============================] - 0s 27ms/step - loss: 6.0669e-04 - accuracy: 1.0000 - val_loss: 3.2274 - val_accuracy: 0.4375
Epoch 64/70
1/1 [==============================] - 0s 30ms/step - loss: 6.3602e-04 - accuracy: 1.0000 - val_loss: 3.2300 - val_accuracy: 0.4375
Epoch 65/70
1/1 [==============================] - 0s 26ms/step - loss: 7.5042e-04 - accuracy: 1.0000 - val_loss: 3.2318 - val_accuracy: 0.4375
Epoc

In [45]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 6016), started 0:45:52 ago. (Use '!kill 6016' to kill it.)

## Using TensorBoard with other methods

In [35]:
#Use the same dataset as above, but convert it to tf.data.Dataset to take advantage of batching capabilities:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [36]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [37]:
#Create stateful metrics that can be used to accumulate values during training and logged at any point:

# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [38]:
#Define the training and test functions:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        train_loss(loss)
        train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [39]:
#Set up summary writers to write the summaries to disk in a different logs directory:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [40]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))
    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 0.24488294124603271, Accuracy: 92.88333129882812, Test Loss: 0.1162695661187172, Test Accuracy: 96.5
Epoch 2, Loss: 0.10446560382843018, Accuracy: 96.87999725341797, Test Loss: 0.08016292005777359, Test Accuracy: 97.41999816894531
Epoch 3, Loss: 0.07211146503686905, Accuracy: 97.80833435058594, Test Loss: 0.07443124055862427, Test Accuracy: 97.66999816894531
Epoch 4, Loss: 0.05525840446352959, Accuracy: 98.27833557128906, Test Loss: 0.06205476075410843, Test Accuracy: 98.02999877929688
Epoch 5, Loss: 0.04428074136376381, Accuracy: 98.54833221435547, Test Loss: 0.05914183333516121, Test Accuracy: 98.1199951171875


In [41]:
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 8952), started 0:09:06 ago. (Use '!kill 8952' to kill it.)